# Run AML code in your custom Conda Environment from base image

Create and register my Environment with custom Conda and python packages from my docker image

## Check if environment already exists

In [2]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Workspace
from azureml.core import Experiment, ScriptRunConfig, Environment

# Name of your Environment
myEnvironmentName = 'demoEnvFromDockerImageWithCustomCondaPythonPackages'

# Get the Workspace
ws = Workspace.from_config()
print(ws.name, "loaded")

try:
    env_name = Environment.get(workspace=ws, name = myEnvironmentName)
    print('Environment {} already exists, stop program'.format(env_name))
except Exception as e:
    print('Environment {} not found. Proceed to create it...'.format(myEnvironmentName))

tbexamaml loaded
Environment Environment(Name: demoEnvFromDockerImageWithCustomCondaPythonPackages,
Version: 1) already exists, stop program


# Create your custom environment using your docker image from your private registry

In [4]:
# Create the environment
env_name = Environment(name=myEnvironmentName)
# Enable Docker and reference an image
env_name.docker.enabled = True
# To use your own installed packages and disable Conda, set the parameter
env_name.python.user_managed_dependencies=True
env_name.docker.base_image = "<your azure container registry name>.azurecr.io/demo-aml/<name of the docker image>:latest"
env_name.docker.base_image_registry.username = "<azure container registry service principal id>"
env_name.docker.base_image_registry.password = "<service principal password>"


# Create or attach a compute target

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Name of the compute target
cluster_name = "CustomAMLCompute"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    # Create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use get_status() to get a detailed status for the current AmlCompute.
print(compute_target.get_status().serialize())

Found existing compute target.
{'errors': [], 'creationTime': '2021-03-03T15:54:32.753905+00:00', 'createdBy': {'userObjectId': 'e41d4ef3-c9c5-4f08-a494-4c3cb9d953aa', 'userTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47', 'userName': 'Tirthankar Barari (TR)'}, 'modifiedTime': '2021-03-03T15:57:24.228416+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS3_V2'}


# Now Train the model from your python script

In [6]:
from azureml.widgets import RunDetails

training_folder = 'diabetes-training'
experiment_name = 'train-diabetes-in-custom-environment'

# Create a script config
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                compute_target=compute_target,
                                environment=env_name) 

# submit the experiment run
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# Show the running experiment run in the notebook widget
RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion(show_output=True)

# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: train-diabetes-in-custom-environment_1616181850_2cb70b34
Web View: https://ml.azure.com/experiments/train-diabetes-in-custom-environment/runs/train-diabetes-in-custom-environment_1616181850_2cb70b34?wsid=/subscriptions/7ee5c9fb-ba32-4243-af24-bda7af98f453/resourcegroups/tbexamaml/workspaces/tbexamaml

Streaming azureml-logs/55_azureml-execution-tvmps_826b8cb4838b93ade22dabb217dc64247e8c4ad564a550fdf2498431fa1b4b55_d.txt

2021-03-19T19:24:31Z Starting output-watcher...
2021-03-19T19:24:31Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-03-19T19:24:31Z Executing 'Copy ACR Details file' on 10.0.0.4
2021-03-19T19:24:31Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
Login Succeeded
latest: Pulling from demo-aml/tr-custom-conda-1.0
Digest: sha256:89690468bce52db07de777d21a600b2c87166c1f57685aa24bfb9e14949fcafd
Status: Image is up to date for tbdemoacr.azurecr.io/demo-aml/tr-custom-conda-1.0:latest
tbdemoacr.azurecr.io/demo-aml/tr-custom-conda-

# Register the custom environment

In [7]:
env_name.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "tbdemoacr.azurecr.io/demo-aml/tr-custom-conda-1.0:latest",
        "baseImageRegistry": {
            "address": null,
            "password": "AzureMlSecret=Env_686800b17fa718ea0fa5c0727ee8cfc618a994aad0f714fa1dbae725315d3709_1#EnvironmentDefinition#ContainerRegistry#Password",
            "registryIdentity": null,
            "username": "f570e9be-6e58-496a-a480-5ea716375618"
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "demoEnvFromDockerImageWithC

# List all environments in the workspace

In [8]:
Environment.list(workspace=ws)

{'myEnvironmentWithCustomCondaImage': {
     "databricks": {
         "eggLibraries": [],
         "jarLibraries": [],
         "mavenLibraries": [],
         "pypiLibraries": [],
         "rcranLibraries": []
     },
     "docker": {
         "arguments": [],
         "baseDockerfile": "./Dockerfile",
         "baseImage": null,
         "baseImageRegistry": {
             "address": null,
             "password": null,
             "registryIdentity": null,
             "username": null
         },
         "enabled": true,
         "platform": {
             "architecture": "amd64",
             "os": "Linux"
         },
         "sharedVolumes": true,
         "shmSize": null
     },
     "environmentVariables": {
         "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
     },
     "inferencingStackVersion": null,
     "name": "myEnvironmentWithCustomCondaImage",
     "python": {
         "baseCondaEnvironment": null,
         "condaDependencies": {
             "channels": [
                 